In [1]:
import itertools
import numpy as np
import pandas as pd
# for data scaling and splitting
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
# for neural net
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# for evaluation
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [2]:
data = pd.read_csv("data/combined_expression.csv")
data.head()

,CELL_LINE_NAME,classification,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,COL15A1,C6orf10,TMEM225,NOTCH4,PBX2,AGER,RNF5,AGPAT1,DFNB59,PRRT1
0,1240121,1,6.419526,3.182094,9.320548,3.759654,3.802619,3.215753,4.698729,7.873672,...,3.245454,2.953508,3.543429,3.352022,4.672310,3.641128,3.135310,3.737072,3.450927,3.168800
1,1240122,2,7.646494,2.626819,10.153853,3.564755,3.942749,3.290760,3.551675,8.252413,...,2.786709,3.077382,3.728232,3.208882,4.586840,3.395654,3.586800,3.519128,3.115323,3.051645
2,1240123,1,8.319417,3.111183,9.643558,4.757258,3.919757,3.602185,3.329644,9.076950,...,3.459089,3.085394,3.462811,3.339030,4.614897,3.395845,3.419193,3.971646,3.729310,3.320022
3,1240124,1,9.006994,3.028173,9.686700,4.280504,3.147646,3.188881,3.293807,8.678790,...,2.835403,2.960303,3.415083,3.290171,4.770123,3.400821,3.383734,3.798107,2.822404,3.297547
4,1240127,1,7.985676,2.694729,10.676134,4.159685,3.804637,3.481942,3.111261,7.555407,...,2.896523,2.849899,3.480114,3.226128,5.832710,3.612179,3.347095,4.457963,5.198524,4.553586


In [3]:
selected_genes = pd.read_csv('cleaned/boruta-99-25-0.01.csv')
selected_genes = selected_genes.values.tolist()
selected_genes = list(itertools.chain(*selected_genes))

In [4]:
# retrieving proper columns
X = data.loc[:, selected_genes]
y = data['classification'].values
# scaling the data
scalar = MinMaxScaler()
x_scaled = scalar.fit_transform(X)
# splitting data (20% test, 80% train)
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=0)

# Gridsearch for Input and Output Layer (no hidden layers)

## Optimizing Epochs and Batches

In [5]:
def create_model():
    model = Sequential()
    # adding layers
    model.add(Dense(len(selected_genes), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [15]:
model = KerasClassifier(build_fn=create_model)
epochs = [10, 25, 50, 100]
batches = [16, 32, 64, 128]
param_grid = dict(epochs=epochs, batch_size=batches)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, verbose=0, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

/Users/eddieguo/anaconda3/envs/fixed_comp/lib/python3.6/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/50
513/513 [==============================] - 0s 972us/step - loss: 0.6632 - accuracy: 0.6238
Epoch 2/50
513/513 [==============================] - 0s 85us/step - loss: 0.5979 - accuracy: 0.6628
Epoch 3/50
513/513 [==============================] - 0s 83us/step - loss: 0.5593 - accuracy: 0.6979
Epoch 4/50
513/513 [==============================] - 0s 84us/step - loss: 0.5484 - accuracy: 0.7115
Epoch 5/50
513/513 [==============================] - 0s 85us/step - loss: 0.5294 - accuracy: 0.7173
Epoch 6/50
513/513 [==============================] - 0s 95us/step - loss: 0.5194 - accuracy: 0.7505
Epoch 7/50
513/513 [==============================] - 0s 97us/step - loss: 0.5465 - accuracy: 0.7037
Epoch 8/50
513/513 [==============================] - 0s 95us/step - loss: 0.5835 - accuracy: 0.6920
Epoch 9/50
513/513 [==============================] - 0s 91us/step - loss: 0.5858 - accuracy: 0.6940
Epoch 10/50
513/513 [==============================] - 0s 89us/step - loss: 0.5838 - accur

In [17]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.754386 using {'batch_size': 64, 'epochs': 50}


In [43]:
grid_result.cv_results_

{'mean_fit_time': array([ 8.89025078, 12.75803318, 19.26391606, 34.30394907,  8.20030737,
        11.34873967, 13.6889308 , 18.04424515,  6.45221438,  8.43450785,
         9.72793574, 14.60181489,  7.19966426,  6.71270518,  6.27273889,
         8.398774  ]),
 'std_fit_time': array([0.16444237, 0.12779544, 0.54525051, 0.20913776, 0.52294618,
        0.51423753, 1.31642935, 0.08993733, 0.33960881, 0.32237099,
        0.4168923 , 0.24949539, 0.47613226, 0.29489586, 0.36781995,
        0.1170051 ]),
 'mean_score_time': array([0.94311066, 0.66564546, 1.10282264, 0.70391574, 0.9941628 ,
        1.18192902, 0.82030702, 0.78884726, 0.9586884 , 0.68544936,
        0.96039133, 0.6519465 , 0.95165958, 0.60001125, 0.47210975,
        0.18315883]),
 'std_score_time': array([0.03113768, 0.02067126, 0.14116664, 0.15304386, 0.22508128,
        0.21197436, 0.17951082, 0.15242896, 0.07337433, 0.09686909,
        0.27478934, 0.0482765 , 0.19803076, 0.03923923, 0.16044839,
        0.04926153]),
 'param_ba

## Tuning the Training Optimization Algorithm

In [47]:
def create_model2(optimizer='adam'):
    model = Sequential()
    # adding layers
    model.add(Dense(len(selected_genes), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compiling
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [48]:
model = KerasClassifier(build_fn=create_model2, epochs=50, batch_size=64)
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5)
grid_result = grid.fit(X_train, y_train)

Epoch 1/50
513/513 [==============================] - 0s 884us/step - loss: 0.6181 - accuracy: 0.6725
Epoch 2/50
513/513 [==============================] - 0s 73us/step - loss: 0.5726 - accuracy: 0.7076
Epoch 3/50
513/513 [==============================] - 0s 74us/step - loss: 0.5564 - accuracy: 0.7251
Epoch 4/50
513/513 [==============================] - 0s 76us/step - loss: 0.5450 - accuracy: 0.7583
Epoch 5/50
513/513 [==============================] - 0s 77us/step - loss: 0.5297 - accuracy: 0.7505
Epoch 6/50
513/513 [==============================] - 0s 76us/step - loss: 0.5395 - accuracy: 0.7271
Epoch 7/50
513/513 [==============================] - 0s 75us/step - loss: 0.5232 - accuracy: 0.7505
Epoch 8/50
513/513 [==============================] - 0s 77us/step - loss: 0.5173 - accuracy: 0.7485
Epoch 9/50
513/513 [==============================] - 0s 77us/step - loss: 0.5356 - accuracy: 0.7563
Epoch 10/50
513/513 [==============================] - 0s 76us/step - loss: 0.5323 - accur

In [49]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.734893 using {'optimizer': 'Adamax'}


# Testing the Model

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=64)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
model.fit(X_train, y_train)
test_predictions = model.predict(X_test)

In [ ]:
print(classification_report(y_test, test_predictions))

In [ ]:
print(confusion_matrix(y_test,test_predictions))

In [ ]:
model.save('model/model_1.h5')